# 📚 Retail Analytics Copilot - Complete Tutorial
## A Beginner's Guide to AI Agents, RAG, LangGraph, and DSPy

---

Welcome! This notebook will teach you **everything** about this project, starting from zero knowledge.

**What you'll learn:**
1. What an AI agent is
2. What RAG (Retrieval-Augmented Generation) means
3. How to query databases with natural language
4. What LangGraph is and why we use it
5. How DSPy optimizes AI systems
6. How all the pieces fit together

Let's start from the very beginning!

---

## 🎯 Part 1: What Problem Are We Solving?

### The Scenario
Imagine you work at a company called **Northwind**. They sell products like beverages, dairy, seafood, etc.

Your boss wants to ask questions like:
- "What was our total revenue in June 1997?"
- "What's the return policy for beverages?"
- "Which product made the most money?"

### The Traditional Problem
Normally, your boss would need to:
1. Find someone who knows SQL (database language)
2. Wait for them to write a query
3. Read through policy documents
4. Combine information from multiple sources

### Our Solution: An AI Agent
We built an **AI agent** (a smart assistant) that can:
- Understand questions in plain English
- Search through documents (policies, KPIs, calendars)
- Write and run SQL queries automatically
- Combine information from both sources
- Give you a typed answer with references

**This is called a "Retail Analytics Copilot"** - it helps you analyze retail data!

---

## 🧩 Part 2: Key Concepts Explained (Simple)

### 1. What is RAG? 🔍

**RAG = Retrieval-Augmented Generation**

Think of it like this:
- **Retrieval**: Finding relevant information (like using Google to search)
- **Augmented**: Adding that information to help the AI
- **Generation**: AI creates an answer using what it found

**Example:**
```
Question: "What's the return policy for beverages?"

Step 1 (Retrieval): Search documents for "return" + "beverages"
  → Finds: "Beverages unopened: 14 days"

Step 2 (Generation): AI reads this and answers:
  → "The return window for unopened beverages is 14 days."
```

### 2. What is an AI Agent? 🤖

An **agent** is an AI that can:
- Make decisions ("Should I search documents or query database?")
- Use tools (search engine, database, calculator)
- Execute multiple steps to solve complex problems
- Fix its own mistakes

It's like having a smart assistant that doesn't just answer questions, but **knows how to find the answers**.

### 3. What is LangGraph? 🕸️

**LangGraph** is a framework for building AI agents as a graph of steps.

Think of it like a flowchart:
```
Start → Classify Question → Search Docs? → Query Database? → Combine Results → Answer
```

Each box is a "node" (a step), and arrows are "edges" (connections).

### 4. What is DSPy? 🎓

**DSPy = Declarative Self-improving Python**

It's a library that helps you:
- Define what you want the AI to do (not how)
- Automatically improve the AI with examples
- Make the AI more reliable

**Analogy:**
- **Regular prompting**: "Please convert this question to SQL"
- **DSPy**: You provide 10 examples, and DSPy teaches the AI to be better at it

### 5. What is BM25? 📊

**BM25 = Best Match 25**

It's a search algorithm (like Google's search, but simpler).

**How it works:**
1. You have documents split into chunks
2. You ask a question
3. BM25 finds which chunks match your question best
4. Returns the top matching chunks

**Example:**
```
Chunks:
  A: "Beverages return policy: 14 days unopened"
  B: "Dairy products expire quickly"
  C: "Winter campaign focuses on confections"

Question: "beverage return"
BM25 Score: A=0.95, B=0.1, C=0.05
→ Returns chunk A
```

---

## 🏗️ Part 3: Project Structure Overview

Our project has these key parts:

```
1. Data Sources
   ├─ Northwind Database (orders, products, customers)
   └─ Documents (policies, KPIs, marketing calendar)

2. Tools
   ├─ SQLite Tool (queries the database)
   └─ Retriever (searches documents with BM25)

3. AI Brain (DSPy Modules)
   ├─ Router (decides what to do)
   ├─ Planner (makes a plan from docs)
   ├─ SQL Generator (writes SQL queries)
   └─ Synthesizer (creates final answer)

4. Workflow (LangGraph)
   └─ Connects all pieces in the right order

5. Optimization (DSPy)
   └─ Makes the SQL Generator smarter
```

Let's explore each part!

---

## 📦 Part 4: Component 1 - The Database

### What is the Northwind Database?

It's a sample database containing:
- **Orders**: What customers bought and when
- **Products**: Items for sale (beverages, dairy, etc.)
- **Customers**: Who's buying
- **Order Details**: Specific items in each order

### How We Access It

We use **SQLite** (a simple database) and **SQL** (a query language).

**Example SQL:**
```sql
SELECT ProductName, UnitPrice 
FROM Products 
WHERE CategoryID = 1
```

This means: "Show me product names and prices where category is 1"

Let's see our SQLite Tool in action:

In [ ]:
# Import our SQLite tool
import sys
sys.path.append('/home/shamaseen/Downloads/just project')

from agent.tools.sqlite_tool import SQLiteTool

# Create the tool
db_tool = SQLiteTool('data/northwind.sqlite')

# Get the database schema (structure)
schema = db_tool.get_schema()
print("Database Tables:")
print(schema[:500])  # Print first 500 characters

In [ ]:
# Let's run a simple query
result = db_tool.execute_query("SELECT COUNT(*) FROM Products")

print("Query Result:")
print(f"Columns: {result['columns']}")
print(f"Data: {result['rows']}")
print(f"Error: {result['error']}")

# Explanation:
# COUNT(*) counts all rows
# We're asking: "How many products are there?"

---

## 📄 Part 5: Component 2 - The Documents

### What Documents Do We Have?

We have 4 markdown files:

1. **marketing_calendar.md**: Campaign dates (e.g., "Summer Beverages 1997: June 1-30")
2. **kpi_definitions.md**: How to calculate metrics (e.g., Average Order Value formula)
3. **catalog.md**: Product categories
4. **product_policy.md**: Return policies (e.g., "Beverages: 14 days")

### How We Search Them

We use **BM25** (a search algorithm) in our **Retriever**.

Let's see it in action:

In [ ]:
from agent.rag.retrieval import Retriever

# Create retriever
retriever = Retriever('docs')

print(f"Loaded {len(retriever.chunks)} document chunks\n")

# Search for information about returns
query = "return policy beverages"
results = retriever.retrieve(query, k=2)

print(f"Top results for '{query}':")
for i, result in enumerate(results, 1):
    print(f"\n{i}. Chunk ID: {result['id']}")
    print(f"   Score: {result['score']:.2f}")
    print(f"   Content: {result['content'][:100]}...")

**What just happened?**

1. We loaded 4 documents and split them into small chunks
2. We searched for "return policy beverages"
3. BM25 found the most relevant chunk
4. It gave us a score (higher = more relevant)

This is **RAG's first step**: Retrieval!

---

## 🧠 Part 6: Component 3 - The AI Brain (DSPy Modules)

### What are DSPy Signatures?

A **Signature** is like a job description for the AI. It says:
- What inputs the AI gets
- What outputs the AI should produce

### Our 4 Signatures:

1. **Router**: Classifies questions
   - Input: Question
   - Output: "rag" or "sql" or "hybrid"

2. **Planner**: Extracts information from docs
   - Input: Question + Retrieved docs
   - Output: Plan (dates, KPIs, constraints)

3. **SQL Generator**: Writes SQL
   - Input: Question + Schema + Plan
   - Output: SQL query

4. **Synthesizer**: Creates final answer
   - Input: Question + SQL results + Retrieved docs
   - Output: Answer + Explanation + Citations

Let's look at the code:

In [ ]:
# View our DSPy signatures
with open('agent/dspy_signatures.py', 'r') as f:
    content = f.read()
    
# Show just the Router signature
print("Example: Router Signature\n")
print(content[content.find("class Router"):content.find("class GenerateSQL")])

**Understanding this code:**

```python
class Router(dspy.Signature):
    """Classify the user question..."""
    
    question = dspy.InputField(...)   # What AI receives
    classification = dspy.OutputField(...)  # What AI produces
```

This is like saying:
- **Job**: Classify questions
- **Input**: A question (text)
- **Output**: A classification ("rag", "sql", or "hybrid")

DSPy handles the rest automatically!

---

## 🕸️ Part 7: Component 4 - The Workflow (LangGraph)

### What is LangGraph's Job?

LangGraph connects all our components in the right order, like a flowchart:

```
┌──────────┐
│  Start   │
└────┬─────┘
     │
     ▼
┌──────────┐
│  Router  │ ← Decides: RAG? SQL? Both?
└────┬─────┘
     │
     ├─ RAG path ────────┐
     ├─ SQL path ──────┐ │
     └─ Hybrid ──────┐ │ │
                     │ │ │
                     ▼ ▼ ▼
               ┌───────────┐
               │ Retriever │ ← Search docs
               └─────┬─────┘
                     │
                     ▼
               ┌───────────┐
               │  Planner  │ ← Extract info
               └─────┬─────┘
                     │
                     ▼
               ┌────────────┐
               │ SQL Gen    │ ← Write query
               └─────┬──────┘
                     │
                     ▼
               ┌────────────┐
               │ Executor   │ ← Run query
               └─────┬──────┘
                     │
                  Error? ───┐
                     │      │
                     ▼      ▼
               ┌────────────┐
               │ Synthesizer│ ← Final answer
               └─────┬──────┘
                     │
                     ▼
                  Answer!
```

### Our 7 Nodes:

1. **Router**: Classify the question
2. **Retriever**: Search documents
3. **Planner**: Extract constraints from docs
4. **SQL Generator**: Write SQL query
5. **Executor**: Run the SQL
6. **Repair**: Fix errors (up to 2 tries)
7. **Synthesizer**: Create final answer

Let's visualize the graph:

In [ ]:
# Show the graph structure (simplified)
print("""Our LangGraph has:

NODES (7 total):
  1. router          - Classify question
  2. retriever       - Search documents
  3. planner         - Extract info from docs
  4. sql_generator   - Write SQL query
  5. executor        - Run SQL query
  6. repair          - Fix errors
  7. synthesizer     - Create final answer

EDGES (connections):
  router → retriever (if RAG or Hybrid)
  router → sql_generator (if SQL)
  retriever → planner
  planner → sql_generator (if Hybrid/SQL)
  planner → synthesizer (if RAG only)
  sql_generator → executor
  executor → synthesizer (if success)
  executor → repair (if error)
  repair → sql_generator (retry)
  synthesizer → END

This creates a smart workflow that adapts based on:
  - Question type (RAG/SQL/Hybrid)
  - Success or failure
  - Number of retries
""")

---

## 🎓 Part 8: Component 5 - DSPy Optimization

### What is Optimization?

Think of teaching a student:
- **Before**: Student guesses how to do math
- **After**: You show 10 examples, student learns the pattern

DSPy does the same for our AI:
- **Before**: AI writes SQL based on general knowledge
- **After**: We show 10 good SQL examples, AI learns to be better

### How We Optimized

We used **BootstrapFewShot**:
1. We provided 10 SQL examples in `train_examples.json`
2. DSPy analyzed what makes them good
3. Created a "template" with 4 best examples
4. Saved it to `agent/optimized_sql_gen.json`

Now when the AI generates SQL, it has these examples as reference!

Let's look at the training data:

In [ ]:
import json

# Load training examples
with open('train_examples.json', 'r') as f:
    examples = json.load(f)

print("Training Examples for SQL Generation:\n")
for i, example in enumerate(examples[:3], 1):  # Show first 3
    print(f"{i}. Question: {example['question']}")
    print(f"   SQL: {example['sql_query']}\n")

**What DSPy learned:**
- How to structure SELECT statements
- How to JOIN tables
- How to use WHERE clauses
- Northwind-specific table names

The AI now uses these patterns when generating new SQL!

---

## 🎬 Part 9: How It All Works Together

### Let's trace a complete example:

**Question:** "What was the total revenue in June 1997?"

**Step-by-step execution:**

```
1. ROUTER (AI Module 1)
   Input: "What was the total revenue in June 1997?"
   Think: This needs database query for revenue + dates from docs
   Output: "hybrid"

2. RETRIEVER (Tool 1)
   Input: "revenue June 1997"
   Search: docs/marketing_calendar.md
   Found: "Summer Beverages 1997: June 1-30"
   Output: [chunk with dates]

3. PLANNER (AI Module 2)
   Input: Question + Retrieved chunks
   Extract: "Date range: 1997-06-01 to 1997-06-30"
   Output: Plan with date constraint

4. SQL GENERATOR (AI Module 3)
   Input: Question + Schema + Plan
   Generate: 
     SELECT SUM(UnitPrice * Quantity * (1 - Discount)) 
     FROM "Order Details" od
     JOIN Orders o ON od.OrderID = o.OrderID
     WHERE o.OrderDate BETWEEN '1997-06-01' AND '1997-06-30'
   Output: SQL query

5. EXECUTOR (Tool 2)
   Input: SQL query
   Run: Execute on northwind.sqlite
   Result: 12345.67
   Output: {columns: [...], rows: [[12345.67]], error: None}

6. SYNTHESIZER (AI Module 4)
   Input: Question + SQL result + Retrieved docs
   Create answer:
     final_answer: 12345.67
     explanation: "Calculated from Order Details for June 1997"
     citations: ["Order Details", "Orders", "marketing_calendar::chunk0"]
   Output: Complete answer

FINAL OUTPUT:
{
  "id": "revenue_june_1997",
  "final_answer": 12345.67,
  "sql": "SELECT SUM(...) FROM ...",
  "confidence": 0.8,
  "explanation": "Calculated total revenue from orders in June 1997",
  "citations": ["Order Details", "Orders", "marketing_calendar::chunk0"]
}
```

### Key Points:
1. **Hybrid approach**: Combined document search + database query
2. **RAG in action**: Retrieved date info from docs, used it in SQL
3. **LangGraph flow**: Each node did its job in sequence
4. **DSPy optimization**: SQL generator used learned patterns
5. **Citations**: Traced exactly where info came from

---

## 🧪 Part 10: Running the Full Agent

Now let's see how to run the complete system.

### Method 1: Command Line (Recommended)

```bash
python run_agent_hybrid.py \
  --batch sample_questions_hybrid_eval.jsonl \
  --out outputs_hybrid.jsonl
```

This will:
1. Load 6 test questions
2. Process each through the LangGraph
3. Save results to `outputs_hybrid.jsonl`

### Method 2: Python Code

We can also run it programmatically:

In [ ]:
# Note: This requires Ollama to be running
# You would need: ollama pull phi3.5:3.8b-mini-instruct-q4_K_M

print("""
To run the full agent, you need:

1. Ollama running:
   ollama serve

2. Model downloaded:
   ollama pull phi3.5:3.8b-mini-instruct-q4_K_M

3. Then run:
   python run_agent_hybrid.py --batch sample_questions_hybrid_eval.jsonl --out outputs_hybrid.jsonl

The agent will process all 6 questions and create outputs_hybrid.jsonl!
""")

---

## 📊 Part 11: Understanding the Results

Let's examine the output format:

In [ ]:
# Load and examine one output
import json

try:
    with open('outputs_hybrid.jsonl', 'r') as f:
        first_line = f.readline()
        result = json.loads(first_line)
    
    print("Example Output:\n")
    print(json.dumps(result, indent=2))
    
    print("\n\nBreakdown:")
    print(f"  ID: {result['id']} - Unique question identifier")
    print(f"  Final Answer: {result['final_answer']} - The typed answer")
    print(f"  SQL: {result['sql'][:50]}... - Query that was run")
    print(f"  Confidence: {result['confidence']} - How confident (0-1)")
    print(f"  Explanation: {result['explanation'][:50]}... - Why this answer")
    print(f"  Citations: {result['citations']} - Where info came from")
    
except FileNotFoundError:
    print("Run the agent first to generate outputs!")
    print("Command: python run_agent_hybrid.py --batch sample_questions_hybrid_eval.jsonl --out outputs_hybrid.jsonl")

### Output Fields Explained:

1. **id**: Matches the input question ID
2. **final_answer**: The actual answer (could be number, text, or object)
3. **sql**: The SQL query that was executed (empty for RAG-only questions)
4. **confidence**: 0.0 to 1.0 score
   - 1.0 = Perfect (no errors)
   - 0.8 = Good (1 repair)
   - 0.6 = OK (2 repairs)
5. **explanation**: Brief description (max 2 sentences)
6. **citations**: List of sources used
   - DB tables: "Orders", "Products", etc.
   - Doc chunks: "kpi_definitions::chunk2", etc.

---

## 🔧 Part 12: Key Files Explained

### Project Files and Their Roles:

```
📁 agent/
  ├─ 📄 graph_hybrid.py         - LangGraph workflow (7 nodes)
  ├─ 📄 dspy_signatures.py      - AI module definitions
  ├─ 📄 optimized_sql_gen.json  - Trained SQL generator
  ├─ 📁 rag/
  │  └─ 📄 retrieval.py         - Document search (BM25)
  └─ 📁 tools/
     └─ 📄 sqlite_tool.py       - Database access

📁 data/
  └─ 📄 northwind.sqlite        - Database (23.5 GB)

📁 docs/
  ├─ 📄 marketing_calendar.md   - Campaign dates
  ├─ 📄 kpi_definitions.md      - Metric formulas
  ├─ 📄 catalog.md              - Product categories
  └─ 📄 product_policy.md       - Return policies

📄 run_agent_hybrid.py          - Main entry point (CLI)
📄 optimize_agent.py            - DSPy training script
📄 train_examples.json          - 10 SQL examples
📄 sample_questions_hybrid_eval.jsonl  - 6 test questions
📄 outputs_hybrid.jsonl         - Generated answers
📄 requirements.txt             - Python dependencies
📄 README.md                    - Documentation
```

---

## 💡 Part 13: Common Concepts Summary

### Quick Reference Guide

| Concept | What It Is | Why We Use It |
|---------|------------|---------------|
| **AI Agent** | A program that makes decisions and uses tools | To autonomously solve complex questions |
| **RAG** | Retrieval + AI generation | To use documents as knowledge source |
| **LangGraph** | Workflow framework with nodes | To organize agent steps in a graph |
| **DSPy** | Declarative AI programming | To optimize AI modules with examples |
| **BM25** | Search algorithm | To find relevant document chunks |
| **SQLite** | Database system | To store structured data (orders, products) |
| **SQL** | Query language | To extract data from database |
| **Signature** | AI task definition | To tell AI what inputs/outputs to expect |
| **Module** | DSPy component | A reusable AI unit (e.g., SQL generator) |
| **Node** | Graph step | One action in the workflow |
| **Edge** | Graph connection | How nodes connect to each other |
| **Optimization** | Training process | Making AI better with examples |
| **Bootstrap** | Learning from examples | Creating better prompts automatically |
| **Citation** | Source reference | Where the answer came from |
| **Confidence** | Trust score | How reliable the answer is |

---

## 🎯 Part 14: The Big Picture

### What We Built:

An **intelligent retail analytics assistant** that:

1. **Understands** questions in natural language
2. **Decides** whether to search documents, query database, or both
3. **Retrieves** relevant information from markdown files
4. **Generates** SQL queries to get data
5. **Executes** queries on the database
6. **Combines** information from multiple sources
7. **Fixes** its own mistakes (repair loop)
8. **Produces** typed answers with citations
9. **Learns** from examples to get better (DSPy optimization)

### Why This Matters:

- **No SQL knowledge needed**: Ask in plain English
- **No document reading**: AI finds info for you
- **Trustworthy**: Shows citations and confidence
- **Self-improving**: Gets better with more examples
- **Resilient**: Fixes its own errors
- **Local**: Runs on your machine, no cloud needed

### Real-World Applications:

1. **Business Analytics**: "What was our Q3 revenue?"
2. **Customer Support**: "What's our return policy for X?"
3. **Research**: "Find all studies about topic Y"
4. **Legal**: "What does contract Z say about clause ABC?"
5. **Healthcare**: "Patient data matching criteria XYZ"

---

## 🚀 Part 15: Next Steps - How to Use This Knowledge

### Understanding Your System:

Now you know:
1. ✅ What each file does
2. ✅ How the workflow works
3. ✅ Why we use RAG, LangGraph, and DSPy
4. ✅ How to run the agent
5. ✅ How to interpret results

### Customizing the System:

**Add your own documents:**
```python
# Just add .md files to docs/ folder
# The retriever will automatically load them!
```

**Add your own database:**
```python
# Point SQLiteTool to your .sqlite file
db_tool = SQLiteTool('your_database.sqlite')
```

**Improve SQL generation:**
```python
# Add more examples to train_examples.json
# Run: python optimize_agent.py
```

### Troubleshooting:

**Check trace log:**
```python
# All execution steps are in:
agent_trace.log
```

**Test components separately:**
```bash
python test_components.py  # Test retriever and DB
python test_graph.py       # Test graph structure
```

---

## 📚 Part 16: Additional Resources

### Learn More:

1. **RAG (Retrieval-Augmented Generation)**
   - [What is RAG?](https://research.ibm.com/blog/retrieval-augmented-generation-RAG)
   - Think: "Google search + AI answer"

2. **LangGraph**
   - [Official Docs](https://python.langchain.com/docs/langgraph)
   - Think: "Flowchart for AI agents"

3. **DSPy**
   - [GitHub](https://github.com/stanfordnlp/dspy)
   - Think: "Teaching AI by example"

4. **BM25**
   - [Explanation](https://en.wikipedia.org/wiki/Okapi_BM25)
   - Think: "Smart search algorithm"

### Project Documentation:

- `README.md` - Quick start guide
- `PROJECT_SUMMARY.md` - Architecture overview
- `OPTIMIZATION_RESULTS.md` - DSPy training results
- `FINAL_RESULTS.md` - Test results and metrics
- `ASSIGNMENT_CHECKLIST.md` - Requirements verification

---

## 🎓 Part 17: Quiz - Test Your Understanding

### Questions:

1. **What does RAG stand for?**
   - A) Random Access Generation
   - B) Retrieval-Augmented Generation ✓
   - C) Rapid AI Generator

2. **What is BM25 used for?**
   - A) Database queries
   - B) Document search ✓
   - C) SQL optimization

3. **How many nodes does our LangGraph have?**
   - A) 5
   - B) 6
   - C) 7 ✓

4. **What does the Router module do?**
   - A) Writes SQL
   - B) Searches documents
   - C) Classifies questions ✓

5. **What is DSPy optimization for?**
   - A) Making the AI smarter with examples ✓
   - B) Speeding up execution
   - C) Reducing memory usage

### Answers:
All marked with ✓ above!

**How did you do?**
- 5/5: Expert level! 🎓
- 3-4/5: Good understanding! 👍
- 1-2/5: Review the concepts again 📖
- 0/5: Start from Part 2! 🚀

---

## 🎉 Congratulations!

You've completed the full tutorial!

### What You Learned:

✅ AI agents and how they work
✅ RAG (Retrieval-Augmented Generation)
✅ LangGraph for workflow management
✅ DSPy for AI optimization
✅ BM25 for document search
✅ SQL and database querying
✅ How all components integrate
✅ How to run and customize the system

### You Can Now:

✅ Explain what the project does
✅ Understand each component's role
✅ Run the complete system
✅ Interpret the results
✅ Troubleshoot issues
✅ Customize for your needs

### Remember:

**This project combines:**
- 📚 Document search (RAG)
- 🗄️ Database queries (SQL)
- 🧠 AI intelligence (LLM)
- 🕸️ Workflow management (LangGraph)
- 🎓 Self-improvement (DSPy)

**To create a powerful analytics assistant!**

---

### 💪 You're now ready to:
1. Build your own AI agents
2. Implement RAG systems
3. Optimize AI with DSPy
4. Design complex workflows

**Keep learning and experimenting!** 🚀